In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/openaifab/fair.git

Cloning into 'fair'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 58 (delta 8), reused 0 (delta 0), pack-reused 28
Unpacking objects: 100% (58/58), done.
Checking out files: 100% (33/33), done.


In [3]:
!pip install ckiptagger

In [4]:
%cd fair
%pwd

/content/fair


'/content/fair'

In [5]:
!wget --no-check-certificate \
    https://storage.googleapis.com/aigodjango2/tf2_lstm_model \
    -O model/tf2_lstm_model

--2020-09-11 05:42:54--  https://storage.googleapis.com/aigodjango2/tf2_lstm_model
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62426472 (60M) [application/octet-stream]
Saving to: ‘model/tf2_lstm_model’

model/tf2_lstm_mode 100%[===================>]  59.53M   172MB/s    in 0.3s    

2020-09-11 05:42:55 (172 MB/s) - ‘model/tf2_lstm_model’ saved [62426472/62426472]



In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from gensim.models import word2vec
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from OpenFabLibrary import JeibaCutWords
from OpenFabLibrary import AppendKeywordCheck

##create word ID mapping and word vector

In [7]:
data_dir = "/".join((".", "data"))

#w2v = word2vec.Word2Vec.load('word2vec_model/CBOW')
w2v = word2vec.Word2Vec.load('word2vec_model/zh.bin')
word2id = {k:i for i, k in enumerate(w2v.wv.vocab.keys())}
id2word = {i:k for k, i in word2id.items()}
print(word2id)
word2id_len = len(word2id)
print('word2id_len:', word2id_len)
print('the last word is:', id2word[word2id_len-1])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


{'平定': 0, '上士': 1, '虎尾': 2, '涉': 3, '第二': 4, '淵源': 5, '机匣': 6, '小腿': 7, '编译': 8, '平安': 9, '蝶泳': 10, '探究': 11, '實並': 12, '薙': 13, '五级': 14, '通志': 15, '宁乡县': 16, '這天': 17, '神為': 18, '礦': 19, '沿著': 20, '多瓦': 21, '纸币': 22, '平家': 23, '河': 24, '湖边': 25, '鑾': 26, '行駛': 27, '看作': 28, '历史风貌': 29, '还要': 30, '金文': 31, '好色': 32, '黎加': 33, '通往': 34, '二部': 35, '路局': 36, '困境': 37, '閩南語': 38, '发射场': 39, '這套': 40, '看似': 41, '升起': 42, '遠': 43, '第代': 44, '应付': 45, '二郎': 46, '至歲': 47, '世纪': 48, '明確': 49, '起诉': 50, '五经': 51, '麻省': 52, '盈利': 53, '略有不同': 54, '至正': 55, '太傅': 56, '鯽': 57, '数百年': 58, '小脑': 59, '至此': 60, '世界冠军': 61, '应为': 62, '第七': 63, '第一': 64, '古利': 65, '瀋': 66, '辊': 67, '第三': 68, '漸漸': 69, '应': 70, '外套': 71, '空穴': 72, '审核': 73, '来源不明': 74, '抱住': 75, '桑德斯': 76, '希羅': 77, '作弊': 78, '第个': 79, '阿塞拜疆': 80, '跟随': 81, '卫生部': 82, '亚基': 83, '逼使': 84, '不景氣': 85, '河是': 86, '幫派': 87, '福音书': 88, '通式': 89, '資格賽': 90, '明确': 91, '吉尔吉斯': 92, '戰機': 93, '拱门': 94, '第九': 95, '八號': 96, '休士頓': 97, '其後在': 98, '課長': 

##AI預測 + 關鍵字檢查

In [8]:
#Load the trained model
model = tf.keras.models.load_model('model/tf2_lstm_model')
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 300)           15030600  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 1032      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 9         
Total params: 15,218,521
Trainable params: 187,921
Non-trainable params: 15,030,600
_________________________________________________________________


In [9]:
input_text = "含500億活菌數及八種益生菌，排便不順，氣味難聞，當心健康拉警報，服用本產品可達到體內環保、增強抵抗力並強化細胞功能，可改善體質、促進新陳代謝、幫助維持消化道機能、促進食慾、開胃，促進腸道蠕動改變細菌叢生態，使排便順暢。"
#input_text = "吃飯可以維持健康"

ad_ID = 0
ad_Name = "測試產品"
ad_Class = 0
ad_Description = input_text

test_data_df = pd.DataFrame({'ID': [ad_ID], 
                            'Name':[ad_Name],
                            'Description':[ad_Description],
                            'Class':[ad_Class]})
        
# 斷詞處理
test_df = JeibaCutWords(test_data_df)

# 關鍵字檢查
test_df['keyword_flag'], keywords_list = AppendKeywordCheck(test_df)
    
# 選取多少詞來當作輸入
PICK_WORDS = 40  # 選前面40個詞當作輸入，這個長度要跟訓練模型的長度一樣

docs_pred_id = []
for doc in test_df['sentence']:
    text = doc[:PICK_WORDS]
    ids = [word2id_len]*PICK_WORDS
    ids[:len(text)] = [word2id[w] if w in word2id else word2id_len for w in text] # <OOV> is 50101
    docs_pred_id.append(ids)

# 轉換後的sequence合併到dataframe    
print(test_df)
test_df['sentence_seq'] = docs_pred_id
print(test_df)

x = test_df['sentence_seq'].tolist()
x_pred = np.array(x)
print(x_pred)

probability = model.predict(x_pred)
print(probability)
if probability > 0.5:
  result = "違法"
else:
  keywords_list = []
  result = "合法"


Building prefix dict from /content/fair/Jeiba/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u61e261749d9b9aaf205c1189a189dc8f.cache
Loading model cost 1.509 seconds.
Prefix dict has been built successfully.


   id                                           sentence  class  keyword_flag
0   0  [含, 500, 億, 菌數, 八種, 益生菌, 排便, 不順, 氣味, 難聞, 當心, 健...      0             1
   id  ...                                       sentence_seq
0   0  ...  [24119, 50101, 8792, 50101, 50101, 50101, 5010...

[1 rows x 5 columns]
[[24119 50101  8792 50101 50101 50101 50101  4567 32756 50101 50101 35032
   4707 13283  1590 13834  4233 20963  2625 50101 32858 34162 31140 25634
  50101  5607  7601 34108 44800 50101 19554 10554 50101 38458  6352  4433
  28421 25302 31965 10554]]
[[0.9125502]]


In [10]:
print("辨識結果: ", result)
print("違規機率: ", probability)
print("違規字詞: ", keywords_list)

辨識結果:  違法
違規機率:  [[0.9125502]]
違規字詞:  ['服用', '增強', '強化', '改善', '開胃']
